In [ ]:
# import os
# import cv2

# DATA_DIR = './data'
# if not os.path.exists(DATA_DIR):
#     os.makedirs(DATA_DIR)

# number_of_classes = 48
# dataset_size = 100

# cap = cv2.VideoCapture(0)
# for j in range(number_of_classes):
#     if not os.path.exists(os.path.join(DATA_DIR, str(j))):
#         os.makedirs(os.path.join(DATA_DIR, str(j)))

#     print('Collecting data for class {}'.format(j))

#     done = False
#     while True:
#         ret, frame = cap.read()
#         cv2.putText(frame, 'Ready? Press "Q" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3,
#                     cv2.LINE_AA)
#         cv2.imshow('frame', frame)
#         if cv2.waitKey(25) == ord('q'):
#             break

#     counter = 0
#     while counter < dataset_size:
#         ret, frame = cap.read()
#         cv2.imshow('frame', frame)
#         cv2.waitKey(25)
#         cv2.imwrite(os.path.join(DATA_DIR, str(j), '{}.jpg'.format(counter)), frame)

#         counter += 1
# cap.release()
# cv2.destroyAllWindows()

In [2]:
import os
import pickle
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)
# Path to the parent directory containing subdirectories '0', '1', '2', etc.
DATA_DIR = 'H:/MotionSpeak/hindi/tata'
data = []
labels = []
for subdir in os.listdir(DATA_DIR):
    subdir_path = os.path.join(DATA_DIR, subdir)
    if os.path.isdir(subdir_path):
        for img_path in os.listdir(subdir_path):
            data_aux = []
            x_ = []
            y_ = []
            img_file_path = os.path.join(subdir_path, img_path)
            
            # Check if the file exists
            if os.path.isfile(img_file_path):
                img = cv2.imread(img_file_path)
                # Check if the image is not empty
                if img is not None:
                    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                    results = hands.process(img_rgb)
                    if results.multi_hand_landmarks:
                        for hand_landmarks in results.multi_hand_landmarks:
                            for i in range(len(hand_landmarks.landmark)):
                                x = hand_landmarks.landmark[i].x
                                y = hand_landmarks.landmark[i].y

                                x_.append(x)
                                y_.append(y)

                            for i in range(len(hand_landmarks.landmark)):
                                x = hand_landmarks.landmark[i].x
                                y = hand_landmarks.landmark[i].y
                                data_aux.append(x - min(x_))
                                data_aux.append(y - min(y_))

                        data.append(data_aux)
                        labels.append(subdir)  # Use the directory name as label
                else:
                    print(f"Skipping empty image: {img_file_path}")
            else:
                print(f"Skipping non-existent file: {img_file_path}")

f = open('datapickle', 'wb')
pickle.dump({'data': data, 'labels': labels}, f)
f.close()


In [3]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np


data_dict = pickle.load(open('./datapickle', 'rb'))
filtered_data = []
filtered_labels = []

# Iterate through the data and labels
for i in range(len(data_dict["data"])):
    if len(data_dict["data"][i]) == 42:
        # Append the data point and label if the size is 42
        filtered_data.append(data_dict["data"][i])
        filtered_labels.append(data_dict["labels"][i])

# Update data_dict with the filtered data and labels
data_dict["data"] = filtered_data
data_dict["labels"] = filtered_labels
data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

model = RandomForestClassifier()

model.fit(x_train, y_train)

y_predict = model.predict(x_test)

score = accuracy_score(y_predict, y_test)

print('{}% of samples were classified correctly !'.format(score * 100))

f = open('model_01.p', 'wb')
pickle.dump({'model': model}, f)
f.close()

98.98989898989899% of samples were classified correctly !


In [4]:
len(data_dict["data"])

1483

In [5]:
len(data_dict['labels'])

1483

In [ ]:
# import pickle
# import cv2
# import mediapipe as mp
# import numpy as np

# model_dict = pickle.load(open('./model_01.p', 'rb'))
# model = model_dict['model']

# cap = cv2.VideoCapture(0)

# mp_hands = mp.solutions.hands
# mp_drawing = mp.solutions.drawing_utils
# mp_drawing_styles = mp.solutions.drawing_styles

# hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# labels_dict = labels ={0:'A',1:'AA',2:'ae',3:'aee',4:'ana',5:'Ba',6:'Bha',7:'Cha', 8:'Chha',9:'Da', 10:'Daa',11:'Dha',12:'Dhha', 13:'e', 14:'ee',15:'Fa',16:'Ga',17:'Gha',18:'Gya',19:'Ha',20:'Hello',21:'iloveyou',22:'Ja',23:'Jha',24:'Ka',25:'Kha',26:'La', 27:'Lae', 28:'Ma',29:'Na',30:'Namaste',31:'no',32:'oo',33:'Pa', 34:'Ra',35:'Sa',36:'Sh',37:'Shaa',38:'Ta',39:'Th',40:'Tha',41:'Thanks',42:'Tta',43:'uu',44:'Uuu',45:'Va',46:'Ya',47:'Yes'}
# while True:

#     data_aux = []
#     x_ = []
#     y_ = []

#     ret, frame = cap.read()

#     H, W, _ = frame.shape

#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

#     results = hands.process(frame_rgb)
#     if results.multi_hand_landmarks:
#         for hand_landmarks in results.multi_hand_landmarks:
#             mp_drawing.draw_landmarks(
#                 frame,  # image to draw
#                 hand_landmarks,  # model output
#                 mp_hands.HAND_CONNECTIONS,  # hand connections
#                 mp_drawing_styles.get_default_hand_landmarks_style(),
#                 mp_drawing_styles.get_default_hand_connections_style())

#         for hand_landmarks in results.multi_hand_landmarks:
#             for i in range(len(hand_landmarks.landmark)):
#                 x = hand_landmarks.landmark[i].x
#                 y = hand_landmarks.landmark[i].y

#                 x_.append(x)
#                 y_.append(y)

#             for i in range(len(hand_landmarks.landmark)):
#                 x = hand_landmarks.landmark[i].x
#                 y = hand_landmarks.landmark[i].y
#                 data_aux.append(x - min(x_))
#                 data_aux.append(y - min(y_))

#         x1 = int(min(x_) * W) - 10
#         y1 = int(min(y_) * H) - 10

#         x2 = int(max(x_) * W) - 10
#         y2 = int(max(y_) * H) - 10

#         prediction = model.predict([np.asarray(data_aux[:42])])

#         predicted_character = labels_dict[int(prediction[0])]

#         cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
#         cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
#                     cv2.LINE_AA)

#     cv2.imshow('frame', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

In [6]:
print('{}% of samples were classified correctly !'.format(score * 100))

98.98989898989899% of samples were classified correctly !


In [7]:
import pickle
import cv2
import mediapipe as mp
import numpy as np
import sys

model_dict = pickle.load(open('./model_01.p', 'rb'))
model = model_dict['model']

cap = cv2.VideoCapture(0)

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.7)

labels_dict = {
    0: 'अ', 1: 'आ', 2: 'ए', 3: ' ऐ', 4: 'ण', 5: 'ब', 6: 'भ', 7: 'च', 8: 'छ', 9: 'ड',
    10: 'द', 11: 'ढ', 12: 'ध', 13: 'इ', 14: 'ई', 15: 'फ', 16: 'ग', 17: 'घ', 18: 'ज्ञ',
    19: 'ह', 20: 'हैलो', 21: 'मैं तुमसे प्यार करता हूँ', 22: 'ज', 23: 'झ', 24: 'क', 25: 'ख', 26: 'ल',
    27: 'ळ', 28: 'म', 29: 'न', 30: 'नमस्ते', 31: 'नहीं', 32: 'औ', 33: 'प', 34: 'र',
    35: 'स', 36: 'क्ष', 37: 'श', 38: 'त', 39: 'थ', 40: 'ठ', 41: 'धन्यवाद', 42: 'ट',
    43: 'उ', 44: 'ऊ', 45: 'व', 46: 'य', 47: 'हाँ'
}

while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux[:42])])

        predicted_character = labels_dict[int(prediction[0])]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Write the output to a file
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write(predicted_character)


NameError: name 'predicted_character' is not defined

In [3]:
while True:
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    if frame is None:  # Check if frame is None
        print("Error reading frame from the camera.")
        break

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        prediction = model.predict([np.asarray(data_aux[:42])])

        predicted_character = labels_dict[int(prediction[0])]

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Write the output to a file
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write(predicted_character)


Error reading frame from the camera.


NameError: name 'predicted_character' is not defined